In [20]:
import numpy as np
import pandas as pd

from sklearn.model_selection import RandomizedSearchCV

from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [2]:
def k_fold_cross_validation(ml_pipeline, X, y, n=5, k=10, score='accuracy'):
   
    cv = RepeatedStratifiedKFold(n_splits = n, 
                                 n_repeats = k, 
                                 random_state = 1)
    n_scores = cross_val_score(ml_pipeline, X, y, 
                               scoring = score, cv = cv, 
                               n_jobs = -1)
    
    return(np.array([np.mean(n_scores), np.std(n_scores)]))

In [3]:
def lablesToBinary(multi_labels):
    
    return multi_labels.apply(lambda x: "nonevent" if x == "nonevent" else "event")

In [4]:
def accuracy_score(y, y_pred):
   
    return np.mean(y == y_pred)

In [5]:
def perplexity_score(true_labels, pred_p):
   
    cond_result = []

    for i in range(0, true_labels.size):
        if true_labels[i] == "nonevent":
            cond_result.append(1 - pred_p[i])
        else:
            cond_result.append(pred_p[i])

    return np.exp( -np.mean( np.log( np.array(cond_result) ) ) )

In [6]:
def predict_npf(classifier, X_tr, y, X_te):

    classifier.fit(X_tr, y)

    classes = pd.Series(classifier.predict(X_te))
    p = pd.DataFrame(classifier.predict_proba(X_te))

    results = []

    for i in range(0, classes.size):
        label = classes[i]
        p_value = 1 - p.loc[i, label]
        results.append([label, p_value])

    answers = pd.DataFrame(results, columns=['class4', 'p'])
    answers['class4'] = mapping_class4[answers['class4']]

    return answers